In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(21)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=100, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [3]:
import sys
sys.getsizeof(data_all)

216607760

In [261]:
sorted(item_subset)

[1,
 21,
 22,
 31,
 102,
 200,
 202,
 203,
 204,
 281,
 284,
 350,
 1152,
 1195,
 1203,
 1204,
 1205,
 1221,
 1225,
 1231,
 1504,
 1615,
 2256,
 2257,
 2258,
 2313,
 2501,
 2521,
 2551,
 2581,
 2721,
 2731,
 2736,
 2867,
 2887,
 3045,
 3046,
 3601,
 3716,
 3717,
 3718,
 3721,
 3723,
 3727,
 3731,
 3732,
 3733,
 3737,
 3738,
 3742,
 3766,
 3767,
 3771,
 3772,
 3773,
 3816,
 3817,
 3821,
 3823,
 4267,
 4272,
 4273,
 4410,
 4411,
 4728,
 4976,
 4977,
 4978,
 5060,
 5068,
 5069,
 5073,
 5074,
 5469,
 5501,
 5581,
 5662,
 5690,
 5771,
 6750,
 7009,
 7041,
 7078,
 7080,
 7108,
 7109,
 7110,
 7113,
 7114,
 7128,
 7129,
 7158,
 7159,
 7575,
 7965,
 7967,
 16000,
 16016,
 16017,
 16018]

### `/train`

In [250]:
import requests
import json

url = 'https://fillet.azurewebsites.net/train/'
data_subset.to_parquet('data_subset.parquet')

files = {'data':open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True,
           'project_id': 9
           }

headers = {
           'Accept-Charset': 'UTF-8'
          }

# payload = json.dumps(payload)
r = requests.post(url, data=payload,  timeout=1200, files=files)

files['data'].close()

In [251]:
payload

{'cv_acc': True, 'project_id': 9}

In [252]:
r.status_code

200

In [253]:
r.json()['result']

'Success'

In [254]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

ValueError: DataFrame constructor not properly called!

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [262]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'1': 4.7,
 '21': 2.95,
 '22': 9.1,
 '31': 2.0,
 '102': 9.9,
 '200': 5.85,
 '202': 1.8,
 '203': 3.85,
 '204': 10.0,
 '281': 5.45,
 '284': 10.25,
 '350': 2.7,
 '1152': 4.0,
 '1195': 3.7,
 '1203': 2.95,
 '1204': 3.65,
 '1205': 3.85,
 '1221': 3.2,
 '1225': 2.65,
 '1231': 4.15,
 '1504': 4.55,
 '1615': 2.85,
 '2256': 8.35,
 '2257': 11.7,
 '2258': 12.4,
 '2313': 8.85,
 '2501': 2.65,
 '2521': 5.1,
 '2551': 8.1,
 '2581': 5.1,
 '2721': 13.2,
 '2731': 7.75,
 '2736': 10.1,
 '2867': 5.55,
 '2887': 7.95,
 '3045': 2.35,
 '3046': 2.95,
 '3601': 6.5,
 '3716': 6.15,
 '3717': 6.85,
 '3718': 7.55,
 '3721': 10.2,
 '3723': 11.6,
 '3727': 10.95,
 '3731': 11.3,
 '3732': 12.0,
 '3733': 12.7,
 '3737': 9.2,
 '3738': 9.9,
 '3742': 10.5,
 '3766': 9.8,
 '3767': 10.5,
 '3771': 9.15,
 '3772': 9.85,
 '3773': 10.55,
 '3816': 9.05,
 '3817': 9.75,
 '3821': 10.75,
 '3823': 12.15,
 '4267': 11.45,
 '4272': 12.5,
 '4273': 13.2,
 '4410': 5.1,
 '4411': 5.9,
 '4728': 13.8,
 '4976': 11.8,
 '4977': 12.5,
 '4978': 13.2,
 '5060': 

In [263]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 8}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)

r = requests.post(url, data=payload, headers=headers)
print(r.status_code)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

200


,Estimated,Actual
Qty_1,562.0,483.0
Qty_102,94.0,82.0
Qty_1152,71.0,38.0
Qty_1195,69.0,56.0
Qty_1203,145.0,157.0
...,...,...
Qty_7158,4.0,4.0
Qty_7159,11.0,7.0
Qty_7575,141.0,155.0
Qty_7965,24.0,19.0
